#     RESULTS OF THE  SET OF SIMULATIONS

## Loading results

In [ ]:
%matplotlib notebook

In [ ]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from thermalspin.data_analysis  import *


In [ ]:
# Insert here the name of the simulation set
setname = "ferro_critic_set"

In [ ]:
final_state_lst, L_lst, t_lst, J_lst, h_lst, T_lst, e_lst, m_lst, snp_lst = load_set_results(setname, load_set_snapshots=False)

In [ ]:
L, H, t_whole, J_whole, T_whole, e_whole, m_whole, final_state, snp = arrange_set_results_LH(L_lst, t_lst, J_lst, h_lst, T_lst, e_lst, m_lst, final_state_lst)
L_num = t_whole.shape[0]
H_num = t_whole.shape[1]
t_num = t_whole.shape[2]

## Global behaviour

In [ ]:
fig = plt.figure(figsize=(8,5))
ax = plt.subplot(111)
for i,j in np.ndindex(L_num, H_num):
    ax.plot(t_whole[i,j], e_whole[i,j], label=f"L = {L[i]}, H = {H[j]}")
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xlabel("Step")
plt.ylabel("Energy")
plt.grid()
plt.show()

In [ ]:
m_magnitude_whole = np.sqrt(np.sum(m_whole**2, axis = 3))

In [ ]:
fig = plt.figure(figsize=(8,5))
ax = plt.subplot(111)
for i,j in np.ndindex(L_num, H_num):
    ax.plot(t_whole[i,j], m_magnitude_whole[i,j], label=f"L = {L[i]}, H = {H[j]}")
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xlabel("Step")
plt.ylabel("Absolute magnetization")
plt.grid()
plt.show()

## Single ensemble analysis

In [ ]:
# Insert here index of the ensemble to be analyzed 
L_idx = 0
H_idx = 0

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = plt.subplot(111)
ax.plot(t_whole[L_idx, H_idx], e_whole[L_idx, H_idx], label=f"L = {L[L_idx]}, H = {H[H_idx]}")
ax.legend()
plt.ylabel("Energy")
plt.xlabel("Steps")
plt.title("Energy")
plt.grid()
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = plt.subplot(111)
ax.plot(t_whole[L_idx, H_idx], m_whole[L_idx, H_idx, :, 0], label = r"$M_x$")
ax.plot(t_whole[L_idx, H_idx], m_whole[L_idx, H_idx, :, 1], label = r"$M_y$")
ax.plot(t_whole[L_idx, H_idx], m_whole[L_idx, H_idx, :, 2], label = r"$M_z$")
plt.legend()
plt.ylabel("Magnetization")
plt.xlabel("Steps")
plt.title("Magnetization")
plt.grid()
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = plt.subplot(111)
ax.plot(t_whole[L_idx, H_idx], np.abs(m_whole[L_idx, H_idx, :, 0]), label = r"$|M_x|$")
ax.plot(t_whole[L_idx, H_idx], np.abs(m_whole[L_idx, H_idx, :, 1]), label = r"$|M_y|$")
ax.plot(t_whole[L_idx, H_idx], np.abs(m_whole[L_idx, H_idx, :, 2]), label = r"$|M_z|$")
plt.legend()
plt.ylabel("Absolute magnetization")
plt.xlabel("Steps")
plt.title("Absolute magnetization")
plt.grid()
plt.show()

In [ ]:
plot_state(final_state[L_idx][H_idx])

In [ ]:
plot_spin_directions(final_state[L_idx][H_idx])

## Set results

In [ ]:
# SELECT WARMUP PERIOD TO LAST UNTIL STEP NUMBER warmup_final_step
warmup_final_step = np.argmax(t_whole[0,0]==1e7)
warmup_final_idx = np.argmax(np.equal(t_whole[0,0], warmup_final_step))
t = t_whole[:, :, warmup_final_idx:]
e = e_whole[:, :, warmup_final_idx:]
m = m_whole[:, :, warmup_final_idx:]

In [ ]:
e_mean = np.mean(e, axis=2)
e_std  = np.sqrt(np.var(e, axis=2))
m_mean = np.mean(m, axis=2)
m_rmsd = np.sqrt(np.var(m, axis=2))
m_magnitude = np.sqrt(np.sum(m**2, axis = 3))
m_magnitude_mean = np.mean(m_magnitude, axis=2)
m_magnitude_var = np.sqrt(np.var(m_magnitude, axis=2))

### Mean energy

In [ ]:
fig = plt.figure(figsize=(8,5))
ax = plt.subplot(111)
for i in np.ndindex(L_num):
    ax.errorbar(H, e_mean[i], yerr=[e_std[i]/2, e_std[i]/2], fmt="o--", label=f"L = {L[i]}")
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xlabel(r"$H_z$")
plt.ylabel("Mean energy")
plt.grid()
plt.show()
fig.savefig("./plots/"+setname+"_energy.svg")

### Mean magnetization magnitude

In [ ]:
fig = plt.figure(figsize=(8,5))
ax = plt.subplot(111)
for i in np.ndindex(L_num):
    ax.errorbar(H, m_magnitude_mean[i], yerr=m_magnitude_var[i]/2,  fmt="o--", label=f"L = {L[i]}")
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xlabel(r"$H_z$")
plt.ylabel("Mean magnetization magnitude")
plt.grid()
plt.show()
fig.savefig("./plots/"+setname+"_energy.svg")

In [ ]:
def cov(M,i,j):
    M_mean = np.mean(M, axis=2)
    ret = np.zeros(shape=(L_num, 1))
    for l,t in np.ndindex(L_num, 1):
        ret[l,t] = np.mean((M[l,t,:,i]-M_mean[l,t,i])*(M[l,t,:,j]-M_mean[l,t,j]))
    return ret

In [ ]:
T=1.445

chi_xx = np.zeros(shape=(L_num, 1))
chi_yy = np.zeros(shape=(L_num, 1))
chi_zz = np.zeros(shape=(L_num, 1))
chi_xy = np.zeros(shape=(L_num, 1))
chi_yz = np.zeros(shape=(L_num, 1))
chi_zx = np.zeros(shape=(L_num, 1))

for i in np.ndindex(L_num):
    chi_xx[i] = cov(m,0,0)[i]/T*L[i]**3
    chi_yy[i] = cov(m,1,1)[i]/T*L[i]**3
    chi_zz[i] = cov(m,2,2)[i]/T*L[i]**3
    chi_xy[i] = cov(m,0,1)[i]/T*L[i]**3
    chi_yz[i] = cov(m,1,2)[i]/T*L[i]**3
    chi_zx[i] = cov(m,2,0)[i]/T*L[i]**3

In [ ]:
fig = plt.figure(figsize=(8,5))
ax = plt.subplot(111)
ax.plot(L, chi_zz, "o--")
plt.xlabel(r"$L$")
plt.ylabel(r"$\chi_{zz}$")
plt.title("System susceptibility")
plt.grid()
plt.show()
fig.savefig("./plots/"+setname+"_susceptibility.svg")

### Critical indices

In [ ]:
slope, intercept, r_value, p_value, std_err = stats.linregress(np.log(L), np.log(chi_zz[:,0]))
gamma_nu = slope
gamma_nu_u = std_err

In [ ]:
fig = plt.figure(figsize=(8,5))
ax = plt.subplot(111)
ax.set_xscale('log')
ax.set_yscale('log')
plt.scatter(L, chi_zz[:,0])
plt.plot(L, np.exp(intercept)*L**(slope), ":")
slope_str="{0:.3}".format(slope)
std_str="{0:.3}".format(std_err)
plt.text(10,65,fr"$slope = {slope_str}\pm{std_str}$", fontsize=11)
plt.grid(True, which="both")
plt.xlabel(r"$L$")
plt.ylabel(r"$\chi_{zz}$")
plt.title(f"T=1.445")
plt.show()
fig.savefig("./plots/"+setname+"_gamma_nu.svg")

In [ ]:
slope, intercept, r_value, p_value, std_err = stats.linregress(np.log(L), np.log(m_magnitude_mean[:,0]))
beta_nu = -slope
beta_nu_u = std_err

In [ ]:
fig = plt.figure(figsize=(8,5))
ax = plt.subplot(111)
ax.set_xscale('log')
ax.set_yscale('log')
plt.scatter(L, m_magnitude_mean[:,0])
plt.plot(L, np.exp(intercept)*L**(slope), ":")

slope_str="{0:.3}".format(slope)
std_str="{0:.3}".format(std_err)
plt.text(15,3.1e-1,fr"$slope = {slope_str}\pm{std_str}$", fontsize=11)

plt.grid(True, which="both")
plt.xlabel(r"$L$")
plt.ylabel(r"$\langle m\rangle$")
plt.title(f"T=1.445")
plt.show()
fig.savefig("./plots/"+setname+"_beta_nu.svg")

### Spatial correlation

In [ ]:
scm = spatial_correlation_matrix(final_state[0][0])
r,c = radial_distribution(scm)

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = plt.subplot(111)
ax.plot(r,c)
plt.xlabel(r"$r$")
plt.ylabel(r"$C(r)$")
plt.title("Spatial correlation")
plt.grid()
plt.show()